![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

### MySQL and MariaDB for Python Developers
# Timing and comparing adapter performance

In this project, you will work with multiple different MySQL adapters for Python.

You will need access to a MySQL installation where you have superuser permissions. If you do not have such access elsewhere, installing to your personal workstation is a good idea.  Alternately, you might wish to use a Docker container for a self-contained installation.  See `https://hub.docker.com/_/mysql` for details on that option. 

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Create the sample table**

This task is not particularly dependent upon the choice of adapter.  You simply want to load in the same sample data that was discussed in the lesson, located in this repository as `data/census-zipcodes-2018.tsv.bz2`.  It is compressed here to make the repository smaller.  Consult the Python `bz2` module or use command line tools to expand it.

The next part will work with this table.  Only the fields that exist in the tab-separated file are necessary; we will not use the geometric type that this lesson showed.  As well, you only need to include the square mile measures, which are the canonical ones from the census.  You may wish to use more ordinary names that the codes in the source data, but the columns may be called whatever you like.  Call your table `project_zipcode_geography`.

In this part, you should also install all of the adapters discussed in this lesson.  If you wish to try out `trio_mysql` or `tormysql` that is great practice as well.  But at the least, install one asynchronous (e.g. `aiomysql`) and one synchronous adapter (`mysql.connector` or `PyMySQL`).

**Solution**

The solution is largely the same as provided in the lesson.

In [1]:
import mysql.connector
user, pwd, db = 'ine_student', 'ine-password', 'ine'
host, port = 'localhost', 3306
conn = mysql.connector.connect(database=db, host=host, user=user, password=pwd, port=port)
cur = conn.cursor()

In [2]:
cur.execute('DROP TABLE IF EXISTS project_zipcode_geography;')

sql_geography = """
CREATE TABLE project_zipcode_geography (
  zipcode CHAR(5), 
  land_area NUMERIC(8, 3),  -- largest zips need 5 to left of decimal
  water_area NUMERIC(8, 3), -- sizes with 3 digits of precision
  lat REAL,      
  lon REAL,
  PRIMARY KEY (zipcode)
);
"""
cur.execute(sql_geography)

In [3]:
%%time
from collections import namedtuple
fields = ('zipcode', 'ALAND', 'AWATER', 'land_area', 'water_area', 'lat', 'lon')
Data = namedtuple('Data', fields)

sql = "INSERT into project_zipcode_geography VALUES (%s, %s, %s, %s, %s);"

with open('data/census-zipcodes-2018.tsv') as fh:
    next(fh)   # discard header line
    for line in fh:
        row = Data(*line.strip().split('\t'))
        cur.execute(sql, (row.zipcode, row.land_area, row.water_area, row.lat, row.lon))

conn.commit()

CPU times: user 1.56 s, sys: 590 ms, total: 2.15 s
Wall time: 5.4 s


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Timing different adapters**

For the small data we are working with, all adapters will be very fast.  The speed advantages of asynchronous adapters will not be shown strongly until you work with much larger data sets than this. And especially once you have many queries "in flight* at the same time. However, your task is to time differences in performance of a relatively expensive query that utilizes nested subqueries.

Are you able to measure better performance using `PyMySQL` or `mysql.connector` versus `aiomysql` for this query? Does multi-threading make a difference within syncrhonous adapters? You may wish to write and time scripts at the command line, since within Jupyter, you need to use the trick discussed to nest an asyncio loop within Jupyter's own loop.  More adventurous students should try much larger data sets and more complex queries that they have access to. 

In [4]:
sql_nested = """
SELECT zipcode
FROM project_zipcode_geography 
WHERE 10*water_area > (
    SELECT avg(land_area) FROM project_zipcode_geography )
AND 2*land_area > (
    SELECT avg(water_area) FROM project_zipcode_geography )
ORDER BY land_area + water_area;
"""

# We need this if we stay inside Jupyter
import nest_asyncio
nest_asyncio.apply()

**A possible solution**

This solution is provided as an inspiration.  More meaningful results require larger data and more complex queries, but those will depend on you obtaining such datasets, and access to PostgreSQL operating under realistic loads.

In [10]:
import pymysql
import mysql.connector
import aiomysql
import asyncio

In [11]:
async def query():
    conn = await aiomysql.connect(db=db, host=host, user=user, password=pwd, port=port)
    cur = await conn.cursor()
    await cur.execute(sql_nested)
    results = await cur.fetchall()
    return results

In [12]:
%%time   
loop = asyncio.get_event_loop()
results = loop.run_until_complete(asyncio.gather(query()))
# This nests the results and extra time because of gather()
print(f"Number of results: {len(results[0])}")                              

Number of results: 1405
CPU times: user 20.6 ms, sys: 4.1 ms, total: 24.7 ms
Wall time: 65 ms


In [13]:
%%time
connA = pymysql.connect(database=db, host=host, user=user, password=pwd, port=port)
curA = connA.cursor()
curA.execute(sql_nested)
results = curA.fetchall()
print(f"Number of results: {len(results)}")

Number of results: 1405
CPU times: user 10.4 ms, sys: 3.2 ms, total: 13.6 ms
Wall time: 53.4 ms


In [14]:
%%time
connB = mysql.connector.connect(database=db, host=host, user=user, password=pwd, port=port)
curB = connB.cursor()
curB.execute(sql_nested)
results = curB.fetchall()
print(f"Number of results: {len(results)}")

Number of results: 1405
CPU times: user 15.8 ms, sys: 0 ns, total: 15.8 ms
Wall time: 54.6 ms


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)